In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

import generate as generate
from single_layer import *

## Experiment 1
- Data Generation: R1 -> R1, with only one activation unit
- model 
    - hidden_dim: [20]
    - lr: 0.01

In [3]:
# Constants
d = 8
M = 4
num = 1
T = 32
N_test = 102400
noise = 0.1

lr = 0.01
hidden_dim = [32, 32, 32]
dropout = 0.5
weight_decay = 0.001

In [34]:
T = 256
M = 8
d = 1
(thetan, an, bn) = generate.generate_single_layer_v2(M, d, num)
# (thetan, an, bn) = (np.array([[-3.68246229]]),  np.array([[[-0.23286501,1.04007303,1.25076011,-0.97359801,-0.98042712, -0.24153641 ,-0.4957292 , -0.46015677]]]), np.array([[-1.54155049]]))
# (thetan, an, bn) = (np.array([[-1]]),  np.array([[[-0.23286501,1.04007303,1.25076011,-0.97359801,-0.98042712, -0.24153641 ,-0.4957292 , -0.46015677]]]), np.array([[-1.54155049]]))

(X, Y_noiseless) = generate.generate_single_data_v2(T, an, bn, thetan)
Y = generate.add_noise(Y_noiseless, noise)
input = X[0]
print(X.shape, Y.shape)
print(np.mean(Y))
print(np.std(Y))

(1, 256, 1) (1, 256)
2.911840254587978
1.9604461925592893


In [50]:
lr = 0.001
(model, epoch_number, best_vloss, train_loss) = train_one_model(
    [160, 160, 160], X[0], Y[0],
    val_ratio=0.2, 
    verbose=True,
    lr=lr, 
    weight_decay=0.0000,
    batchNorm=False,
    dropout=0,
    batch_size=64,
    patience=2000, 
    epochs=2000,
)
# print(f"epochs: {epoch_number}, train loss: {train_loss} validation loss: {best_vloss}")
# plt.plot(*zip(*sorted(zip(X[0], predicted))))
# plt.legend(["real"] + ["predicted"])
# plt.show()
model.eval()
(X_test, Y_test) = generate.generate_single_data_v2(N_test, an, bn, thetan)
predicted = model(torch.Tensor(X_test[0])).detach().numpy()
kl_divergence = generate.kl_divergence(predicted.reshape(-1), Y_test.reshape(-1), noise)
print(kl_divergence)
# print(generate.kl_divergence(Y_test, 0, noise))
# print(kl_divergence / generate.kl_divergence(Y_test, 0, noise))


EPOCH 1:
LOSS train 12.38424301147461 valid 8.14673900604248
EPOCH 2:
LOSS train 9.671828985214233 valid 5.859202861785889
EPOCH 3:
LOSS train 6.575093626976013 valid 4.294494152069092
EPOCH 4:
LOSS train 4.696604132652283 valid 5.206929683685303
EPOCH 5:
LOSS train 4.622782766819 valid 3.9535083770751953
EPOCH 6:
LOSS train 3.5506924986839294 valid 2.492321491241455
EPOCH 7:
LOSS train 2.98261296749115 valid 1.9030157327651978
EPOCH 8:
LOSS train 2.3517839908599854 valid 1.4668092727661133
EPOCH 9:
LOSS train 1.7355658411979675 valid 1.2841368913650513
EPOCH 10:
LOSS train 1.3543411195278168 valid 0.878836452960968
EPOCH 11:
LOSS train 1.0157194286584854 valid 0.719620943069458
EPOCH 12:
LOSS train 0.8462473452091217 valid 0.6448299884796143
EPOCH 13:
LOSS train 0.7082808315753937 valid 0.5915749073028564
EPOCH 14:
LOSS train 0.6351059004664421 valid 0.46414023637771606
EPOCH 15:
LOSS train 0.5562591925263405 valid 0.3741767108440399
EPOCH 16:
LOSS train 0.48745880275964737 valid 0.34

In [59]:
model

MultiLayer(
  (fcs): ModuleList(
    (0): Linear(in_features=1, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=128, bias=True)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): Linear(in_features=128, out_features=1, bias=True)
  )
  (dropout): Dropout(p=0, inplace=False)
  (act): LeakyReLU(negative_slope=0.01)
)

In [ ]:
sum(p.numel() for p in model.parameters())

34305

In [123]:
import hyperopt
from hyperopt import fmin, hp, STATUS_OK, Trials
import logging

loggers_to_shut_up = [
    "hyperopt.tpe",
    "hyperopt.fmin",
    "hyperopt.pyll.base",
]
for logger in loggers_to_shut_up:
    logging.getLogger(logger).setLevel(logging.ERROR)

In [203]:
space={
    'lr': hp.loguniform('lr', np.log(1e-5), np.log(1e-2)),
    'weight_decay': hp.loguniform('weight_decay', np.log(1e-5), np.log(1e-1)),
    'dropout': hp.loguniform('dropout', np.log(1e-7), np.log(.5)),
    'hidden_dim': hp.choice('hidden_dim', [ 
        [12, 12, 12],
        [16, 16, 16],
        [32, 32, 32],
        [64, 64, 64],
        [128, 128, 128],
    ]),
}
trials = Trials()
def train_hp(args): 
    print(args)
    (model, epoch_number, best_vloss, train_loss) = train_one_model(
        x=X[0], 
        y=Y[0], 
        val_ratio=0.2, 
        batchNorm=False,
        batch_size=64,
        patience=200, 
        epochs=1000,
        **args,
    )
    model.eval()
    (X_test, Y_test) = generate.generate_single_data_v2(N_test, an, bn, thetan)
    predicted = model(torch.Tensor(X_test)).detach().numpy()
    kl_divergence = generate.kl_divergence(predicted.reshape(-1), Y_test, noise)
    print(kl_divergence[0])
    return {
        "loss": kl_divergence[0],
        "status": STATUS_OK,
        "args": args,
    }

In [204]:
best = fmin(
    train_hp,
    space=space,
    max_evals=10,
    trials=trials,
    show_progressbar=False,
)
print(best)

{'dropout': 0.004918053862550941, 'hidden_dim': (12, 12, 12), 'lr': 1.2111600620952753e-05, 'weight_decay': 0.0002475013233738106}
624.2200361648648
{'dropout': 0.0007985941560865539, 'hidden_dim': (32, 32, 32), 'lr': 0.002598738128092957, 'weight_decay': 0.010686645080930007}
139.045444334311
{'dropout': 1.967596020174811e-05, 'hidden_dim': (16, 16, 16), 'lr': 0.0002622172301826283, 'weight_decay': 3.590534586908799e-05}
111.96444245735262
{'dropout': 0.06939407879941736, 'hidden_dim': (64, 64, 64), 'lr': 0.0011573925653253305, 'weight_decay': 0.022344896671097524}
128.61536018511657
{'dropout': 2.3027271499085934e-07, 'hidden_dim': (64, 64, 64), 'lr': 1.9566964154102838e-05, 'weight_decay': 2.7342172724972617e-05}
170.435347181293
{'dropout': 0.1064278877825801, 'hidden_dim': (16, 16, 16), 'lr': 0.004784405567501814, 'weight_decay': 0.0004382579003536056}
110.18508096148905
{'dropout': 1.5826273224872307e-07, 'hidden_dim': (16, 16, 16), 'lr': 0.004800880945968021, 'weight_decay': 6.3

In [205]:
trials.best_trial["result"]["loss"]

83.19502458654605

In [206]:
trials.best_trial["result"]["args"]

{'dropout': 0.0016559867748346595,
 'hidden_dim': (12, 12, 12),
 'lr': 0.002411035920871128,
 'weight_decay': 0.006066653677299191}